In [1]:
from pyspark.sql import SparkSession


# .config("spark.sql.shuffle.partitions", 20) \  # Optimize for parallelism
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()


sc = spark.sparkContext

fileName = './brewery_data_complete_extended.csv'

# deerfootRDD = (sc.textFile(fileName, 8))
# print('\n'.join(deerfootRDD.zipWithIndex().map(lambda x: str(x[1]) + ': ' + str(x[0])).take(15)))

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer

# Load a CSV file
df = spark.read.csv(fileName, header=True, inferSchema=True)


24/11/18 11:12:16 WARN Utils: Your hostname, MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.219 instead (on interface en0)
24/11/18 11:12:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/18 11:12:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:

# Prepare data
targetColumn = "Beer_Style"
labelColumn = "label"
assembler = VectorAssembler(inputCols=["pH_Level", "Temperature"], outputCol="features")
data = assembler.transform(df).select("features", targetColumn)
indexer = StringIndexer(inputCol=targetColumn, outputCol=labelColumn)
indexed_df = indexer.fit(df).transform(df)
data = assembler.transform(indexed_df).select("features", labelColumn)

print(data.head())

# Split into training and testing
train, test = data.randomSplit([0.7, 0.3])

# # Train a model
# lr = LinearRegression(featuresCol="features", labelCol="targetColumn")
# model = lr.fit(train)

# Initialize Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol=labelColumn,
        maxIter=50,
        regParam=0.1,
        elasticNetParam=0.8,
        predictionCol="prediction"
        )

# Train the model
model = lr.fit(train)

# Evaluate
predictions = model.transform(test)
predictions.show()

# Initialize evaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol=labelColumn,
    predictionCol="prediction",
    metricName="accuracy"
)

# Calculate accuracy
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy = {accuracy:.2f}")

Row(features=DenseVector([5.2898, 24.2043]), label=7.0)


+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[4.50000126083050...|  2.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50000622342672...|  1.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50002129320163...|  3.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50003152645682...|  6.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50003842627774...|  1.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50004397879550...|  5.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50004477361177...|  6.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50005039533961...|  2.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.50005564557337...|  2.0|[3.26122317422683...|[0.12504074868100...|       2.0|
|[4.500060034159

Test Accuracy = 0.12
